In [1]:
import pandas as pd
import numpy as np
import itertools

# Visualización de grafos
from pyvis.network import Network

# Interacciones en el notebook
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [2]:
# Load Data
artworks_data = pd.read_csv('/data/Prado_artworks_wikidata.csv')
interactions = pd.read_csv('/data/Prado_emotions.csv')

# Creo diccionarios para obtener imagenes y títulos de los cuadros
artwork_ids = np.unique(interactions.artworkId.values)
artworks_dict = dict()
images_dict = dict()
for art_id in artwork_ids:
    title = artworks_data[artworks_data['ID'] == art_id]['Title'].values[0]
    image = artworks_data[artworks_data['ID'] == art_id]['Image URL'].values[0]
    artworks_dict[art_id] = title
    images_dict[art_id] = image
    
# Create array of emotions and its colors
colors = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4', '#46f0f0', '#f032e6', '#bcf60c', '#fabebe', '#008080', '#e6beff', '#9a6324', '#fffac8', '#800000', '#aaffc3', '#808000', '#ffd8b1', '#000075', '#808080']
emotions = np.unique(interactions['emotion'].values)
emotions_colors = dict()
for i in range(len(emotions)):
    emotions_colors[emotions[i]] = colors[i]
    
emotions_options = list(emotions)
emotions_options.append(None)

In [3]:
def get_edges(emotion=None):
    
    edges = []
    aux_df = interactions
    
    if emotion != None:
        aux_df = interactions[interactions['emotion'] == emotion]
        
    for inter in aux_df.values:
        user = inter[0]
        artwork = inter[1]
        emotion = inter[2]
        polarity = inter[3]
        edges.append((str(user), artwork, emotion, polarity))
        
    return edges

In [4]:
@interact_manual
def show_graph_bipartite(emotion=emotions_options, show_physics=False):
    users = np.unique(interactions['userId'].values)
    artworks = np.unique(interactions['artworkId'].values)
    
    edges = get_edges(emotion)
    
    vis = Network(notebook=True, width="1000px", height="800px")

    for u in users:
        vis.add_node(str(u), group=0, size=1)

    for a in artworks:
        vis.add_node(artworks_dict[a], group=1, size=30, shape='image', image=images_dict[a])
        
    for e in edges:
        vis.add_edge(e[0], artworks_dict[e[1]], emotion=e[2], polarity=e[3], color=emotions_colors[e[2]])
        
    if show_physics:
        vis.show_buttons(filter_=['physics'])
        
    return vis.show('graph_bipartite.html') 

interactive(children=(Dropdown(description='emotion', options=('agreeableness', 'anger', 'anticipation', 'arro…